In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Breast Cancer Project/new_master_dataset.csv")
df

Mounted at /content/drive


,mag,path,filename,class,slide_id,tumor_type
0,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-001.png,benign,22549CD,A
1,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-002.png,benign,22549CD,A
2,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-003.png,benign,22549CD,A
3,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-004.png,benign,22549CD,A
4,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-005.png,benign,22549CD,A
...,...,...,...,...,...,...
7904,400,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_M_PC-14-15704-400-031.png,malignant,15704,PC
7905,400,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_M_PC-14-15704-400-032.png,malignant,15704,PC
7906,400,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_M_PC-14-15704-400-033.png,malignant,15704,PC
7907,400,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_M_PC-14-15704-400-034.png,malignant,15704,PC


In [2]:
groupd_df = df.groupby("tumor_type")

# You can then perform various operations on the grouped data, such as getting the count of each tumor type
tumor_type_counts = groupd_df.size()
tumor_type_counts

tumor_type
A      444
DC    3451
F     1014
LC     626
MC     792
PC     560
PT     453
TA     569
dtype: int64

In [3]:
# df_train_100 = pd.read_csv("/content/drive/MyDrive/Breast Cancer Project/Mag100/train_df_100.csv")
# df_test_100 = pd.read_csv("/content/drive/MyDrive/Breast Cancer Project/Mag100/test_df_100.csv")
# df_val_100 = pd.read_csv("/content/drive/MyDrive/Breast Cancer Project/Mag100/val_df_100.csv")

In [6]:
!pip install timm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from PIL import Image
import os
from transformers import EfficientNetImageProcessor, EfficientNetForImageClassification
import timm

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Define constants
data_dirs = ["TA", "PT", "PC", "MC", "LC", "F", "DC", "A"]
data_root = "/content/drive/MyDrive/Breast Cancer Project/IW/40"  # Replace with the root directory of your data
train_split = 0.7

# Create a list to store the paths and labels of all images
all_data = []

# Populate the list with paths and labels
for label, folder in enumerate(data_dirs):
    folder_path = os.path.join(data_root, folder)
    image_files = os.listdir(folder_path)
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        all_data.append((image_path, label))

# Split data into training and testing sets
train_data, test_data = train_test_split(all_data, train_size=train_split, shuffle=True, random_state=42)

# Define custom dataset class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        img = Image.open(img_path).convert('RGB')  # Open image and convert to RGB mode
        if self.transform:
            img = self.transform(img)
        label_tensor = torch.tensor(label, dtype=torch.long)  # Convert label to tensor
        return img, label_tensor

# Image preprocessing with augmentation for training
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(90),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor()
])

# Image preprocessing without augmentation for testing and validation
test_val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Create custom datasets
train_dataset = CustomDataset(train_data, transform=train_transform)
test_dataset = CustomDataset(test_data, transform=test_val_transform)

# DataLoaders for batching and shuffling
batch_size = 60  # Define the batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

import timm

# Create model
model = timm.create_model("hf_hub:timm/regnety_160.tv2_in1k", pretrained=True)
model.to(device)  # Move model to GPU

# Define optimizer and scheduler
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=3, verbose=True)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False)
    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        # Ensure the input tensor is passed correctly
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * labels.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        progress_bar.set_postfix({'Loss': train_loss / total, 'Accuracy': 100 * correct / total})

    train_loss = train_loss / len(train_loader.dataset)
    train_accuracy = 100 * correct / total

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  # Move data to GPU
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * labels.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(test_loader.dataset)
    val_accuracy = 100 * correct / total

    print(f'Epoch {epoch+1}/{num_epochs}, '
          f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

    # Adjust learning rate
    scheduler.step(val_loss)

# Test the model
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_accuracy = 100 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.2f}%')

Using device: cuda


Epoch 1/10, Train Loss: 4.7897, Train Accuracy: 26.72%, Val Loss: 2.1167, Val Accuracy: 51.59%


Epoch 2/10, Train Loss: 0.9900, Train Accuracy: 74.14%, Val Loss: 0.6824, Val Accuracy: 76.46%


Epoch 3/10, Train Loss: 0.3199, Train Accuracy: 89.54%, Val Loss: 0.5168, Val Accuracy: 81.64%


Epoch 4/10, Train Loss: 0.1923, Train Accuracy: 93.34%, Val Loss: 0.3398, Val Accuracy: 87.15%


...

Epoch 6/10, Train Loss: 0.1004, Train Accuracy: 96.49%, Val Loss: 0.3022, Val Accuracy: 89.65%


Epoch 7/10, Train Loss: 0.0736, Train Accuracy: 97.21%, Val Loss: 0.2330, Val Accuracy: 90.82%


Epoch 8/10, Train Loss: 0.0682, Train Accuracy: 97.49%, Val Loss: 0.2316, Val Accuracy: 92.32%


Epoch 9/10, Train Loss: 0.0668, Train Accuracy: 97.35%, Val Loss: 0.2194, Val Accuracy: 91.99%


Epoch 10/10, Train Loss: 0.0486, Train Accuracy: 98.50%, Val Loss: 0.2078, Val Accuracy: 92.65%
Test Accuracy: 92.65%


In [7]:
import numpy as np
from sklearn.metrics import classification_report

# Evaluate the model on the test set
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Convert predictions and true labels to numpy arrays
predictions = np.array(predictions)
true_labels = np.array(true_labels)

# Print classification report with three digits
print(classification_report(true_labels, predictions, target_names=data_dirs, digits=3))

              precision    recall  f1-score   support

          TA      0.857     0.973     0.911        37
          PT      0.867     0.788     0.825        33
          PC      1.000     0.843     0.915        51
          MC      0.958     0.907     0.932        75
          LC      0.853     0.744     0.795        39
           F      0.893     0.957     0.924        70
          DC      0.936     0.969     0.952       255
           A      0.975     1.000     0.987        39

    accuracy                          0.927       599
   macro avg      0.917     0.898     0.905       599
weighted avg      0.927     0.927     0.926       599



In [8]:
# Train the model
num_epochs = 20
for epoch in range(10, num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False)
    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        # Ensure the input tensor is passed correctly
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * labels.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        progress_bar.set_postfix({'Loss': train_loss / total, 'Accuracy': 100 * correct / total})

    train_loss = train_loss / len(train_loader.dataset)
    train_accuracy = 100 * correct / total

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  # Move data to GPU
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * labels.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(test_loader.dataset)
    val_accuracy = 100 * correct / total

    print(f'Epoch {epoch+1}/{num_epochs}, '
          f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

    # Adjust learning rate
    scheduler.step(val_loss)

# Test the model
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_accuracy = 100 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.2f}%')

Epoch 11/20, Train Loss: 0.0436, Train Accuracy: 98.28%, Val Loss: 0.2097, Val Accuracy: 92.49%


Epoch 12/20, Train Loss: 0.0455, Train Accuracy: 97.85%, Val Loss: 0.2322, Val Accuracy: 91.65%


Epoch 13/20, Train Loss: 0.0427, Train Accuracy: 97.99%, Val Loss: 0.2099, Val Accuracy: 92.32%


Epoch 14/20, Train Loss: 0.0314, Train Accuracy: 98.78%, Val Loss: 0.2309, Val Accuracy: 91.65%
Epoch 00014: reducing learning rate of group 0 to 1.0000e-05.


Epoch 15/20, Train Loss: 0.0274, Train Accuracy: 98.78%, Val Loss: 0.2337, Val Accuracy: 91.49%


Epoch 16/20, Train Loss: 0.0316, Train Accuracy: 98.71%, Val Loss: 0.2265, Val Accuracy: 91.99%


Epoch 17/20, Train Loss: 0.0335, Train Accuracy: 98.50%, Val Loss: 0.2147, Val Accuracy: 92.49%


Epoch 18/20, Train Loss: 0.0305, Train Accuracy: 98.64%, Val Loss: 0.2164, Val Accuracy: 92.15%
Epoch 00018: reducing learning rate of group 0 to 1.0000e-06.


Epoch 19/20, Train Loss: 0.0294, Train Accuracy: 98.50%, Val Loss: 0.2200, Val Accuracy: 91.99%


Epoch 20/20, Train Loss: 0.0354, Train Accuracy: 98.35%, Val Loss: 0.2162, Val Accuracy: 91.99%
Test Accuracy: 91.99%


In [10]:
import numpy as np
from sklearn.metrics import classification_report

data_dirs = ["TA", "PT", "PC", "MC", "LC", "F", "DC", "A", "None"]

# Evaluate the model on the test set
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Convert predictions and true labels to numpy arrays
predictions = np.array(predictions)
true_labels = np.array(true_labels)

# Print classification report with three digits
print(classification_report(true_labels, predictions, target_names=data_dirs, digits=3))

              precision    recall  f1-score   support

          TA      0.860     1.000     0.925        37
          PT      0.900     0.818     0.857        33
          PC      1.000     0.843     0.915        51
          MC      0.931     0.893     0.912        75
          LC      0.711     0.821     0.762        39
           F      0.918     0.957     0.937        70
          DC      0.945     0.937     0.941       255
           A      1.000     1.000     1.000        39
        None      0.000     0.000     0.000         0

    accuracy                          0.920       599
   macro avg      0.807     0.808     0.805       599
weighted avg      0.925     0.920     0.921       599



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
# Train the model
num_epochs = 30
for epoch in range(20, num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False)
    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        # Ensure the input tensor is passed correctly
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * labels.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        progress_bar.set_postfix({'Loss': train_loss / total, 'Accuracy': 100 * correct / total})

    train_loss = train_loss / len(train_loader.dataset)
    train_accuracy = 100 * correct / total

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  # Move data to GPU
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * labels.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(test_loader.dataset)
    val_accuracy = 100 * correct / total

    print(f'Epoch {epoch+1}/{num_epochs}, '
          f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

    # Adjust learning rate
    scheduler.step(val_loss)

# Test the model
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_accuracy = 100 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.2f}%')

Epoch 21/30, Train Loss: 0.0343, Train Accuracy: 98.07%, Val Loss: 0.2211, Val Accuracy: 91.99%


Epoch 22/30, Train Loss: 0.0275, Train Accuracy: 98.71%, Val Loss: 0.2144, Val Accuracy: 91.82%
Epoch 00022: reducing learning rate of group 0 to 1.0000e-07.


Epoch 23/30, Train Loss: 0.0373, Train Accuracy: 98.42%, Val Loss: 0.2148, Val Accuracy: 92.15%


Epoch 24/30, Train Loss: 0.0336, Train Accuracy: 98.42%, Val Loss: 0.2063, Val Accuracy: 92.65%


Epoch 25/30, Train Loss: 0.0330, Train Accuracy: 98.35%, Val Loss: 0.2110, Val Accuracy: 92.65%


Epoch 26/30, Train Loss: 0.0262, Train Accuracy: 98.78%, Val Loss: 0.2145, Val Accuracy: 92.15%


Epoch 27/30, Train Loss: 0.0253, Train Accuracy: 98.93%, Val Loss: 0.2130, Val Accuracy: 92.15%


Epoch 28/30, Train Loss: 0.0267, Train Accuracy: 98.85%, Val Loss: 0.2111, Val Accuracy: 92.82%
Epoch 00028: reducing learning rate of group 0 to 1.0000e-08.


Epoch 29/30, Train Loss: 0.0324, Train Accuracy: 98.50%, Val Loss: 0.2144, Val Accuracy: 91.99%


Epoch 30/30, Train Loss: 0.0291, Train Accuracy: 98.78%, Val Loss: 0.2199, Val Accuracy: 91.99%
Test Accuracy: 91.99%


In [12]:
import numpy as np
from sklearn.metrics import classification_report

data_dirs = ["TA", "PT", "PC", "MC", "LC", "F", "DC", "A", "None"]

# Evaluate the model on the test set
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Convert predictions and true labels to numpy arrays
predictions = np.array(predictions)
true_labels = np.array(true_labels)

# Print classification report with three digits
print(classification_report(true_labels, predictions, target_names=data_dirs, digits=3))

              precision    recall  f1-score   support

          TA      0.881     1.000     0.937        37
          PT      0.903     0.848     0.875        33
          PC      1.000     0.824     0.903        51
          MC      0.932     0.907     0.919        75
          LC      0.727     0.821     0.771        39
           F      0.930     0.943     0.936        70
          DC      0.934     0.937     0.935       255
           A      1.000     1.000     1.000        39
        None      0.000     0.000     0.000         0

    accuracy                          0.920       599
   macro avg      0.812     0.809     0.809       599
weighted avg      0.924     0.920     0.921       599



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
